In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torch.autograd import *
import torchvision
import torchvision.transforms as transforms
import os
import argparse
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
lr = 0.01

In [3]:

import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=2):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18(num_classes=2):
    return ResNet(BasicBlock, [2, 2, 2, 2])


def ResNet34():
    return ResNet(BasicBlock, [3, 4, 6, 3])


def ResNet50():
    return ResNet(Bottleneck, [3, 4, 6, 3])

def ResNet101():
    return ResNet(Bottleneck, [3, 4, 23, 3])


def ResNet152():
    return ResNet(Bottleneck, [3, 8, 36, 3])


def test():
    net = ResNet18()
    y = net(torch.randn(1, 3, 32, 32))
    print(y.size())

In [4]:

# Data Preparation
print('==> Preparing CIFAKE data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Custom dataset class for CIFAKE
class CIFAKE(torchvision.datasets.ImageFolder):
    def __init__(self, root, transform=None):
        super(CIFAKE, self).__init__(root, transform=transform)
        # Map class names to indices
        self.class_to_idx = {'REAL': 0, 'FAKE': 1}
        self.idx_to_class = {0: 'REAL', 1: 'FAKE'}

# Load CIFAKE dataset
train_path = '/kaggle/input/cifake-real-and-ai-generated-synthetic-images/train'
test_path = '/kaggle/input/cifake-real-and-ai-generated-synthetic-images/test'

trainset = CIFAKE(root=train_path, transform=transform_train)
testset = CIFAKE(root=test_path, transform=transform_test)

trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

==> Preparing CIFAKE data..


In [ ]:
# Add these imports at the top of your file
import time
from tqdm import tqdm

# Model
print('==> Building model..')
net = ResNet18(num_classes=2)  # 2 classes for REAL and FAKE
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

# Training and Test Functions with progress bars
def train(epoch, net):
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    start_time = time.time()
    
    with tqdm(trainloader, unit="batch") as tepoch:
        tepoch.set_description(f"Epoch {epoch+1}")
        for batch_idx, (inputs, targets) in enumerate(tepoch):
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            
            # Update progress bar
            tepoch.set_postfix(loss=loss.item(), accuracy=100.*correct/total)
    
    epoch_time = time.time() - start_time
    print(f"Epoch {epoch+1} training time: {epoch_time:.2f} seconds")
    return train_loss/len(trainloader)

def test(epoch, net):
    global acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    start_time = time.time()
    
    with torch.no_grad():
        with tqdm(testloader, unit="batch") as tepoch:
            tepoch.set_description(f"Test {epoch+1}")
            for batch_idx, (inputs, targets) in enumerate(tepoch):
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = net(inputs)
                loss = criterion(outputs, targets)

                test_loss += loss.item()
                _, predicted = outputs.max(1)
                total += targets.size(0)
                correct += predicted.eq(targets).sum().item()
                
                # Update progress bar
                tepoch.set_postfix(loss=loss.item(), accuracy=100.*correct/total)
    
    test_time = time.time() - start_time
    print(f"Test {epoch+1} time: {test_time:.2f} seconds")
    acc = 100 * correct / total
    return test_loss/len(testloader)

# Training Loop
train_losses = []
test_losses = []
epochs = 5

total_start_time = time.time()
for epoch in range(epochs):
    train_losses.append(train(epoch, net))
    test_losses.append(test(epoch, net))
    scheduler.step()

total_time = time.time() - total_start_time
print(f'Total training time: {total_time:.2f} seconds')
print('Accuracy of the network on the test images: %d %%' % (acc))

# Plot training curves
plt.figure(figsize=(10, 5))
plt.plot(np.arange(1, epochs+1), train_losses, label='Train Loss')
plt.plot(np.arange(1, epochs+1), test_losses, label='Test Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Test Loss Curves')
plt.savefig('training_curves.png')
plt.show()

# Confusion Matrix
def plot_confusion_matrix(model, data_loader, device):
    model.eval()
    all_preds = []
    all_targets = []
    start_time = time.time()
    
    with torch.no_grad():
        with tqdm(data_loader, unit="batch") as tepoch:
            tepoch.set_description("Creating confusion matrix")
            for images, labels in data_loader:
                images = images.to(device)
                labels = labels.to(device)
                
                outputs = model(images)
                _, preds = torch.max(outputs, 1)
                
                all_preds.extend(preds.cpu().numpy())
                all_targets.extend(labels.cpu().numpy())
    
    cm_time = time.time() - start_time
    print(f"Confusion matrix generation time: {cm_time:.2f} seconds")
    
    cm = confusion_matrix(all_targets, all_preds)
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['REAL', 'FAKE'], 
                yticklabels=['REAL', 'FAKE'])
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.savefig('confusion_matrix.png')
    plt.show()

plot_confusion_matrix(net, testloader, device)

# Save the model
print("==> Saving model...")
model_save_time = time.time()
torch.save({
    'model_state_dict': net.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'scheduler_state_dict': scheduler.state_dict(),
}, 'resnet18_model.pth')
print(f"Model saved in {time.time() - model_save_time:.2f} seconds")

==> Building model..


Epoch 1: 100%|██████████| 782/782 [06:33<00:00,  1.99batch/s, accuracy=89, loss=0.0626]  


Epoch 1 training time: 393.07 seconds


Test 1: 100%|██████████| 200/200 [01:18<00:00,  2.55batch/s, accuracy=91.7, loss=0.163] 


Test 1 time: 78.49 seconds


Epoch 2:  38%|███▊      | 299/782 [00:30<00:51,  9.38batch/s, accuracy=92.8, loss=0.167] 

In [ ]:
torch.save({
    'epoch': epochs,
    'model_state_dict': net.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'scheduler_state_dict': scheduler.state_dict(),
    'train_losses': train_losses,
    'test_losses': test_losses,
}, 'resnet18_final.pth')
print("Final model saved as resnet18_final.pth")

In [11]:
from torchsummary import summary

summary(net, input_size=(3, 32, 32))  

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
            Conv2d-3           [-1, 64, 32, 32]          36,864
       BatchNorm2d-4           [-1, 64, 32, 32]             128
            Conv2d-5           [-1, 64, 32, 32]          36,864
       BatchNorm2d-6           [-1, 64, 32, 32]             128
        BasicBlock-7           [-1, 64, 32, 32]               0
            Conv2d-8           [-1, 64, 32, 32]          36,864
       BatchNorm2d-9           [-1, 64, 32, 32]             128
           Conv2d-10           [-1, 64, 32, 32]          36,864
      BatchNorm2d-11           [-1, 64, 32, 32]             128
       BasicBlock-12           [-1, 64, 32, 32]               0
           Conv2d-13          [-1, 128, 16, 16]          73,728
      BatchNorm2d-14          [-1, 128,

In [ ]:
from tqdm import tqdm
import time

# Modify the FGSM attack function to show progress (if used in loops)
def FGSM(net, x, y, eps):
    x_ = Variable(x.data, requires_grad=True)
    h_ = net(x_)
    cost = criterion(h_, y)
    net.zero_grad()
    cost.backward()

    pert = eps * x_.grad.detach().sign()
    x_adv = x_ + pert

    h_adv = net(x_adv)
    _, y_adv = torch.max(h_adv.data, 1)
    return x_adv, h_adv, y_adv, pert

epochs = 3

optimizer_adv = optim.SGD(net_adv.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
scheduler_adv = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_adv, T_max=200)

# Adversarial Training with FGSM - add progress bar
def train_adv(epoch, net):
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    eps = 8/255
    start_time = time.time()
    
    with tqdm(trainloader, unit="batch") as tepoch:
        tepoch.set_description(f"Adv Epoch {epoch+1}")
        for batch_idx, (inputs, targets) in enumerate(tepoch):
            inputs, targets = inputs.to(device), targets.to(device)

            inputs_ = Variable(inputs.data, requires_grad=True)
            h_ = net(inputs_)
            cost = criterion(h_, targets)
            net.zero_grad()
            cost.backward()

            pert = eps * inputs_.grad.detach().sign()
            x_adv = inputs_ + pert

            optimizer_adv.zero_grad()
            outputs = net_adv(x_adv)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer_adv.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            
            # Update progress bar
            tepoch.set_postfix(loss=loss.item(), accuracy=100.*correct/total)
    
    epoch_time = time.time() - start_time
    print(f"Adv Epoch {epoch+1} time: {epoch_time:.2f} seconds")
    return train_loss/len(trainloader)

# Train adversarial model with progress
train_losses_adv = []
test_losses_adv = []
total_start_time = time.time()

for epoch in range(epochs):
    train_losses_adv.append(train_adv(epoch, net))
    test_losses_adv.append(test(epoch, net_adv))
    scheduler_adv.step()

total_time = time.time() - total_start_time
print(f'Total adversarial training time: {total_time:.2f} seconds')
print('Accuracy of adversarially-trained network: %d %%' % (acc))

# Evaluate against FGSM attacks with progress
def test_adv(net, net_adv, eps):
    net.train()
    net_adv.eval()
    correct = 0
    total = 0
    start_time = time.time()
    
    with tqdm(testloader, unit="batch") as tepoch:
        tepoch.set_description(f"FGSM Test (ε={eps:.4f})")
        for batch_idx, (inputs, targets) in enumerate(tepoch):
            inputs, targets = inputs.to(device), targets.to(device)
            x_adv, _, _, _ = FGSM(net, inputs, targets, eps)
            
            with torch.no_grad():
                outputs = net_adv(x_adv)
                _, predicted = outputs.max(1)
                total += targets.size(0)
                correct += predicted.eq(targets).sum().item()
                
            # Update progress bar
            tepoch.set_postfix(accuracy=100.*correct/total)
    
    test_time = time.time() - start_time
    print(f"FGSM test (ε={eps:.4f}) time: {test_time:.2f} seconds")
    return 100 * correct / total

print("Evaluating against FGSM attacks:")
for eps in [4/255, 8/255, 12/255]:
    accuracy = test_adv(net, net_adv, eps)
    print(f"Epsilon: {eps:.4f}, Accuracy: {accuracy:.2f}%")

In [ ]:

epochs = 5


# PGD Attack Function with progress
def PGD(net, x, y, alpha, epsilon, iter):
    delta = torch.zeros_like(x, requires_grad=True)
    for i in range(iter):
        loss = criterion(net(x + delta), y)
        loss.backward()
        delta.data = (delta + x.shape[0]*alpha*delta.grad.data).clamp(-epsilon, epsilon)
        delta.grad.zero_()
    pert = delta.detach()
    x_adv = x + pert
    h_adv = net(x_adv)
    _, y_adv = torch.max(h_adv.data, 1)
    return x_adv, h_adv, y_adv, pert

# Adversarial Training with PGD - add progress bar
def train_pgd(epoch, net, alpha, epsilon, iter):
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    start_time = time.time()
    
    with tqdm(trainloader, unit="batch") as tepoch:
        tepoch.set_description(f"PGD Epoch {epoch+1}")
        for batch_idx, (inputs, targets) in enumerate(tepoch):
            inputs, targets = inputs.to(device), targets.to(device)
            x_adv, _, _, _ = PGD(net, inputs, targets, alpha, epsilon, iter)

            optimizer_pgd.zero_grad()
            outputs = net_pgd(x_adv)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer_pgd.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            
            # Update progress bar
            tepoch.set_postfix(loss=loss.item(), accuracy=100.*correct/total)
    
    epoch_time = time.time() - start_time
    print(f"PGD Epoch {epoch+1} time: {epoch_time:.2f} seconds")
    return train_loss/len(trainloader)

# Adversarial Training with PGD
print('==> Building PGD adversarial model..')
net_pgd = ResNet18(num_classes=2).to(device)
if device == 'cuda':
    net_pgd = torch.nn.DataParallel(net_pgd)
    cudnn.benchmark = True

optimizer_pgd = optim.SGD(net_pgd.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
scheduler_pgd = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_pgd, T_max=200)
# Train PGD model with progress
train_losses_pgd = []
test_losses_pgd = []
alpha = 3/255
epsilon = 8/255
iter = 3
total_start_time = time.time()

for epoch in range(epochs):
    train_losses_pgd.append(train_pgd(epoch, net_pgd, alpha, epsilon, iter))
    test_losses_pgd.append(test(epoch, net_pgd))
    scheduler_pgd.step()

total_time = time.time() - total_start_time
print(f'Total PGD training time: {total_time:.2f} seconds')
print('Accuracy of PGD-trained network: %d %%' % (acc))


# Evaluate PGD model against PGD attacks with progress
def test_pgd(net, net_pgd, alpha, eps, iter):
    net.train()
    net_pgd.eval()
    correct = 0
    total = 0
    start_time = time.time()
    
    with tqdm(testloader, unit="batch") as tepoch:
        tepoch.set_description(f"PGD Test (iter={iter})")
        for batch_idx, (inputs, targets) in enumerate(tepoch):
            inputs, targets = inputs.to(device), targets.to(device)
            x_adv, _, _, _ = PGD(net, inputs, targets, alpha, eps, iter)

            with torch.no_grad():
                outputs = net_pgd(x_adv)
                _, predicted = outputs.max(1)
                total += targets.size(0)
                correct += predicted.eq(targets).sum().item()
                
            # Update progress bar
            tepoch.set_postfix(accuracy=100.*correct/total)
    
    test_time = time.time() - start_time
    print(f"PGD test (iter={iter}) time: {test_time:.2f} seconds")
    return 100 * correct / total

print("Evaluating against PGD attacks:")
for iter in [3, 7, 12]:
    accuracy = test_pgd(net, net_pgd, alpha, epsilon, iter)
    print(f"Iterations: {iter}, Accuracy: {accuracy:.2f}%")

# Save models with progress indication
print("Saving models...")
start_time = time.time()
torch.save(net.state_dict(), 'natural_model.pth')
torch.save(net_adv.state_dict(), 'fgsm_model.pth')
torch.save(net_pgd.state_dict(), 'pgd_model.pth')
print(f"Models saved in {time.time() - start_time:.2f} seconds")